# RAG

Para este ejemplo se mejorará el ejemplo que se creó antes ([aquí](../01-rag/rag.ipynb)), se utilizará la misma fuente de información, pero ahora siguiendo el correcto paso a paso de cómo se debería hacer en realidad (ver [aquí](../rag-explanation.md)). Para este caso, utilizaremos la libreria `langchain` con Claude como LLM, y bases de datos de vectores para almacenar la información de los documentos

## Importar módulos y todo lo necesario

Para el modelo de embedding se puede cambiar a otro en caso de ser necesario

In [77]:
# Tipado
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.documents import Document
from langchain_core.messages import AIMessage

# LLM
from langchain_anthropic import ChatAnthropic

# Para el indexing
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_nomic import NomicEmbeddings
from langchain_chroma import Chroma

# Para la generación
from langchain import hub

# Variables de entorno
from os import getenv
from dotenv import load_dotenv

load_dotenv()

CLAUDE_API_KEY = getenv("CLAUDE_API_KEY")
MODEL_NAME = getenv("CLAUDE_MODEL_NAME")
NOMIC_API_KEY = getenv("NOMIC_API_KEY")
LANGCHAIN_API_KEY = getenv("LANGCHAIN_API_KEY")

CHUNK_SIZE = int(getenv("CHUNK_SIZE"))
CHUNK_OVERLAP = int(getenv("CHUNK_OVERLAP"))

In [73]:
llm: ChatAnthropic = ChatAnthropic(
    api_key=CLAUDE_API_KEY,
    model=MODEL_NAME,
    temperature=0,
    max_tokens=2000,
)

text_splitter: RecursiveCharacterTextSplitter = RecursiveCharacterTextSplitter(
    chunk_size=CHUNK_SIZE, chunk_overlap=CHUNK_OVERLAP
)

embeddings: NomicEmbeddings = NomicEmbeddings(
    model="nomic-embed-text-v1.5",
    nomic_api_key=NOMIC_API_KEY
)

vector_store: Chroma = Chroma(embedding_function=embeddings)

prompt: ChatPromptTemplate = hub.pull("rlm/rag-prompt", api_key=LANGCHAIN_API_KEY)

## Indexación

### Document Loader

In [60]:
loader: TextLoader = TextLoader("./info.md", "utf-8")
docs = loader.load()

### Text Splitter

In [61]:
all_splits: list[Document] = text_splitter.split_documents(docs)
all_splits[:3]

[Document(metadata={'source': './info.md'}, page_content='# NEBULA\n\n## Roadmap\n\n### Purpose and Scope of the SAD\n\nEl SAD tiene como objetivo documentar la arquitectura de Nebula, detallando los componentes estructurales y sus interacciones para cumplir con los requisitos de desarrollo, escalabilidad y seguridad. En este documento se incluye toda la informaci�n sobre la arquitectura de software de Nebula, destacando aquellos aspectos que influyen en el rendimiento y la adaptabilidad del sistema. Las decisiones de dise�o documentadas en el SAD son aquellas que tienen un impacto arquitect�nico relevante en la estructura general del sistema, mientras que los detalles de implementaci�n espec�ficos se documentan en otros informes t�cnicos.\n\n### How the SAD Is Organized'),
 Document(metadata={'source': './info.md'}, page_content='El SAD de Nebula est� estructurado en las siguientes secciones:  \nSección 1: Documentation Roadmap - Proporciona una visi�n general del SAD, el prop�sito de

### Vector Store

In [62]:
store = vector_store.add_documents(all_splits)

In [63]:
store[:3]

['663ae405-b1bd-499a-ae0b-97115c7a0f36',
 '68f0cf62-669f-45ba-a435-2fc2b705c949',
 'aa37f863-481c-4b62-87ad-c185f76ada9e']

## Recuperación y Generación

Con `"rlm/rag-prompt"`, se está haciendo este prompt al modelo:

```
You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: {question} 
Context: {context} 
Answer:
```

Ver [esto](https://smith.langchain.com/hub/rlm/rag-prompt) para más info


En este caso, para la generación de la respuesta, no se utilizará LangGraph, pero también es posible

In [ ]:
from typing import TypedDict


class State(TypedDict):
    question: str
    context: list[Document]
    answer: str

In [93]:
def generate(question: str, return_response: bool = False) -> State | (tuple[State, AIMessage]):
    retrieved_docs = vector_store.similarity_search(question)
    docs_content = "<context>\n\n"
    docs_content += "\n\n".join(f"<context{i+1}>{doc.page_content}</context{i+1}>" 
                               for i, doc in enumerate(retrieved_docs))
    docs_content += "\n\n</context>"

    message_prompt = {"question": question, "context": docs_content}
    messages = prompt.invoke(message_prompt)
    response = llm.invoke(messages)

    output = {
            "question": question,
            "context": retrieved_docs,
            "answer": response.content
        }
    
    if not return_response:
        return output
    
    return output, response

## Pruebas

Aún queda hacer la creación más estrictas de pruebas, y hacer cambios

In [96]:
def show_answers(question: str) -> None:
    response = generate(question)

    print("### Question ###")
    print(response["question"])

    print("\n\n### Context ###")
    for i, doc in enumerate(response["context"]):
        print(f"<context{i+1}>")
        print(doc)
        print(f"</context{i+1}>\n")

    print("\n### Answer ###")
    print(response["answer"])


In [97]:
show_answers("¿Cuáles son los objetivos principales del proyecto Nebula?")

### Question ###
¿Cuáles son los objetivos principales del proyecto Nebula?


### Context ###
<context1>
page_content='Nebula est� dise�ado para ser escalable y modular, permitiendo una f�cil integraci�n de nuevas funcionalidades y mejoras a medida que evolucionen las necesidades de los usuarios.

#### Goals and Context

##### Objetivos del Proyecto

El objetivo principal de Nebula es proporcionar un sistema SaaS accesible y eficiente para la creaci�n y evaluaci�n de modelos de aprendizaje autom�tico, espec�ficamente para tareas de regresi�n y clasificaci�n. La plataforma est� dise�ada para usuarios con diversos niveles de experiencia en ciencia de datos, brindando una experiencia intuitiva y automatizada que permite crear modelos precisos sin necesidad de conocimientos avanzados en machine learning.

##### Objetivos Espec�ficos' metadata={'source': './info.md'}
</context1>

<context2>
page_content='Nebula est� dise�ado para ser escalable y modular, permitiendo una f�cil integraci�n de

In [98]:
show_answers("Qué tecnologías se tuvieron en cuenta para Nebula?")

### Question ###
Qué tecnologías se tuvieron en cuenta para Nebula?


### Context ###
<context1>
page_content='##### Contexto del Proyecto

Nebula surge en un contexto donde la demanda por herramientas accesibles de machine learning est� en crecimiento. En la actualidad, el uso de machine learning es com�n en sectores como el financiero, sanitario, retail, y de tecnolog�a, donde se requiere tanto la predicci�n de valores continuos (regresi�n) como la clasificaci�n en categor�as.
La plataforma busca posicionarse como una herramienta que, adem�s de ser f�cil de usar, brinde un alto grado de flexibilidad y precisi�n. El proyecto utiliza tecnolog�as de desarrollo web modernas (React, FastAPI, PostgreSQL) y herramientas de aprendizaje autom�tico (Scikit-learn), lo cual permite que Nebula mantenga un rendimiento �ptimo y una experiencia de usuario fluida.

##### Usuarios Objetivo' metadata={'source': './info.md'}
</context1>

<context2>
page_content='##### Contexto del Proyecto

Nebula surge

In [99]:
show_answers("Para qué sirvió el SAD en Nebula?")

### Question ###
Para qué sirvió el SAD en Nebula?


### Context ###
<context1>
page_content='# NEBULA

## Roadmap

### Purpose and Scope of the SAD

El SAD tiene como objetivo documentar la arquitectura de Nebula, detallando los componentes estructurales y sus interacciones para cumplir con los requisitos de desarrollo, escalabilidad y seguridad. En este documento se incluye toda la informaci�n sobre la arquitectura de software de Nebula, destacando aquellos aspectos que influyen en el rendimiento y la adaptabilidad del sistema. Las decisiones de dise�o documentadas en el SAD son aquellas que tienen un impacto arquitect�nico relevante en la estructura general del sistema, mientras que los detalles de implementaci�n espec�ficos se documentan en otros informes t�cnicos.

### How the SAD Is Organized' metadata={'source': './info.md'}
</context1>

<context2>
page_content='# NEBULA

## Roadmap

### Purpose and Scope of the SAD

El SAD tiene como objetivo documentar la arquitectura de Nebul